<a href="https://colab.research.google.com/github/nichakornchaisuwan/Project_Boneage/blob/main/data_loader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
#Install sitk library to read dicom files
!pip install SimpleITK  
import pandas as pd
import os
import SimpleITK as sitk
import torch
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
import numpy as np
train = pd.read_csv(r"/content/drive/MyDrive/Project_Boneage/train.csv")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 16.5 MB/s eta 0:00:00


In [3]:
train

,id,boneage,male
0,1377,180,False
1,1378,12,False
2,1379,94,False
3,1380,120,True
4,1381,82,False
...,...,...,...
10084,13259,94,False
10085,13261,94,False
10086,13274,94,False
10087,13286,94,False


In [4]:
train['id'] = train['id'].apply(lambda x: str(x)+'.png')
#train['boneage'] = train['boneage'].apply(lambda x: str(x))
train['male'] = train['male'].apply(lambda x: 'male' if x else 'female')

df1 = pd.DataFrame(train)
df1.rename(columns={'male':'gender'},inplace=True)


In [5]:
train

,id,boneage,gender
0,1377.png,180,female
1,1378.png,12,female
2,1379.png,94,female
3,1380.png,120,male
4,1381.png,82,female
...,...,...,...
10084,13259.png,94,female
10085,13261.png,94,female
10086,13274.png,94,female
10087,13286.png,94,female


In [6]:
train.dtypes

id         object
boneage     int64
gender     object
dtype: object

In [6]:
#df = df.drop(['Sex','SmokingStatus'],axis=1)

In [7]:
from torch.autograd import Variable

In [26]:
def read_image(path):
  reader = sitk.ImageSeriesReader()

  names = reader.GetGDCMSeriesFileNames(path)
  reader.SetFileNames(names)

  image = reader.Execute()

  image = sitk.GetArrayFromImage(image)
  #some images in the dataset had a different size, this would have to be 
  #pre-processed using registration or simply cropping as done below
  image = image[0:20,0:256,0:256]
  
  return(image)

class CombineDataset(Dataset):

    def __init__(self, frame, id_col, label_name, path_imgs, transform=None):
        """
        Args:
            frame (pd.DataFrame): Frame with the tabular data.
            id_col (string): Name of the column that connects image to tabular data
            label_name (string): Name of the column with the label to be predicted
            path_imgs (string): path to the folder where the images are.
            transform (callable, optional): Optional transform to be applied
                on a sample, you need to implement a transform to use this.
        """
        self.frame = frame
        self.id_col = id_col
        self.label_name = label_name
        self.path_imgs = path_imgs
        #self.transform = transform

    def __len__(self):
        return (self.frame.shape[0])

    def __getitem__(self, x):
        if torch.is_tensor(x):
            x = x.tolist()
        #complete image path and read
        img_name = self.frame[self.id_col].iloc[x]
        path = os.path.join(self.path_imgs,img_name)
        image = read_image(path)
        image = Variable(torch.Tensor(image)) #torch.from_numpy(image.astype(np.float32))

        #get the other features to be used as training data
        feats = [feat for feat in self.frame.columns if feat not in [self.label_name,self.id_col]]
        feats  = np.array(self.frame[feats].iloc[x])
        feats = Variable(torch.Tensor(feats)) #torch.from_numpy(feats.astype(np.float32))
       
        
        #get label
        label = np.array(self.frame[self.label_name].iloc[x])
        label = Variable(torch.Tensor(label)) #torch.from_numpy(label.astype(np.float32))
        #label = torch.as_tensor(np.array(label).astype('float'))

        return image, feats, label

In [27]:
path_imgs = r"/content/drive/MyDrive/Project_Boneage/boneage_training_dataset"
train_set = CombineDataset(train,'id','boneage','gender',path_imgs)

loader_trainer = DataLoader(
    train_set,
    batch_size = 2,
    shuffle = True,
    num_workers = 0,
    drop_last=True
)


In [23]:
for data in loader_trainer:
    x,z, y = data
    
    print('Image Loaded Size = ', x.shape)
    print('Tabular data = ', z)
    print('Label = ', y)


RuntimeError: ignored